<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/alex-works/Colabs/Grids_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


time: 46.3 s (started: 2023-10-03 15:39:19 +00:00)


In [2]:
%%capture
! git clone https://github.com/alessandronascimento/pyLiBELa.git
! ( cd pyLiBELa && git checkout alex-works)
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2


time: 2min 13s (started: 2023-10-03 15:40:05 +00:00)


In [1]:
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [2]:
from google.colab import drive
import os
from tqdm import tqdm
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = True
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 60 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-10;
Input.min_delta = 1E-5;
Input.dock_min_tol = 1E-10;
search_box = 20.0 #@param {type:"number"}
Input.timeout = 30 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

In [4]:
src_path = 'drive/MyDrive/WorkingProj/PDBbind_v2020/targets_refined_set' #@param {type:"string"}
out_path = 'drive/MyDrive/WorkingProj/PDBbind_v2020/grids'

In [5]:
def calculate_grid(lig_src, rec_src, input, writer, out_prefix, log_path):
  try:
    rec = Mol2(input, rec_src)
    lig = Mol2(input, lig_src)

    coord = COORD_MC()
    hb = FindHB()

    for i in range(len(rec.residue_pointer)-1):
      hb.parse_residue(rec.residue_pointer[i]-1, rec.residue_pointer[i+1]-2, rec.resnames[i], rec, lig, 9.0)
      hb.find_ligandHB(lig_src, lig)

    center = coord.compute_com(lig)

    input.grid_prefix = out_prefix

    grid = Grid(input, writer, rec, center)
    grid.write_grids_to_file()

  except Exception as err:
    with open(f'{log_path}/error_log.txt', 'a') as f:
      f.write(f'ligand/receptor failed at {lig_src.split("/")[-2]} with error: {err}\n')


In [ ]:
writer = WRITER(Input)

done = {name.split('.')[0] : None for name in os.listdir(out_path)}
print(len(done))

for i in tqdm(os.listdir(src_path)):
  if i in done: continue
  calculate_grid(f'{src_path}/{i}/{i}_ligand.mol2.gz', f'{src_path}/{i}/{i}_rec.mol2.gz', Input, writer, f'{out_path}/{i}', out_path)

458


 18%|█▊        | 974/5316 [1:23:05<17:57:32, 14.89s/it]